In [5]:
import json
import pandas as pd
from pathlib import Path
import csv
from typing import Dict, Any, List, Tuple
import random

def print_section(title, char="=", width=80):
    """Print formatted section header."""
    print(f"\n{char * width}")
    print(title.center(width))
    print(f"{char * width}\n")

def find_column_index(df: pd.DataFrame, column_name: str) -> int:
    """Find the index of a column by name."""
    if column_name in df.columns:
        return df.columns.get_loc(column_name)
    raise ValueError(f"Column '{column_name}' not found in dataframe")

def get_numerical_bin_sentence(value: float, edges: List, sentences: List[str]) -> str:
    """
    Find which bin the value falls into and return corresponding sentence.
    edges = [0, 20, 50, 80, "inf"] creates 4 bins:
      - bin 0: [0, 20)
      - bin 1: [20, 50)
      - bin 2: [50, 80)
      - bin 3: [80, inf)
    """
    # Convert "inf" strings to float('inf')
    edges_numeric = []
    for e in edges:
        if isinstance(e, str) and e.lower() in ['inf', '-inf']:
            edges_numeric.append(float(e))
        else:
            edges_numeric.append(float(e))
    
    # Find bin index
    bin_index = 0
    for i in range(len(edges_numeric) - 1):
        if value >= edges_numeric[i] and value < edges_numeric[i + 1]:
            bin_index = i
            break
        elif i == len(edges_numeric) - 2:  # Last bin (to infinity)
            bin_index = i
    
    return sentences[bin_index]

def ensure_period(text: str) -> str:
    """Ensure text ends with a period."""
    text = text.strip()
    if text and not text.endswith('.'):
        text += '.'
    return text

def generate_sentence(column_name: str, value: Any, schema_field: Dict) -> str:
    """
    Generate a full sentence for a column value based on schema rules.
    Returns: "{title}: {sentence}"
    """
    # Skip null/missing values
    if pd.isna(value):
        return None
    
    title = schema_field.get('title', column_name)
    field_type = schema_field.get('type')
    
    if field_type == 'nominal':
        # Match value to code_map
        code_map = schema_field.get('code_map') or {}
        sentence = code_map.get(str(value))
        if sentence:
            return f"{title}: {ensure_period(sentence)}"
    
    elif field_type == 'nl':
        # Use raw value as sentence
        return f"{title}: {ensure_period(str(value))}"
    
    elif field_type == 'numerical':
        # Use semantic bins
        semantic_bins = schema_field.get('semantic_bins', {})
        edges = semantic_bins.get('edges', [])
        sentences = semantic_bins.get('sentences', [])
        
        if edges and sentences:
            try:
                sentence = get_numerical_bin_sentence(float(value), edges, sentences)
                return f"{title}: {ensure_period(sentence)}"
            except (ValueError, IndexError) as e:
                print(f"    ⚠️  Error processing numerical value {value}: {e}")
                return None
    
    elif field_type == 'numerical_coded':
        # Try code_map first, fallback to semantic bins
        code_map = schema_field.get('code_map') or {}
        sentence = code_map.get(str(value))
        
        if sentence:
            return f"{title}: {ensure_period(sentence)}"
        
        # Fallback to semantic bins
        semantic_bins = schema_field.get('semantic_bins', {})
        edges = semantic_bins.get('edges', [])
        sentences = semantic_bins.get('sentences', [])
        
        if edges and sentences:
            try:
                sentence = get_numerical_bin_sentence(float(value), edges, sentences)
                return f"{title}: {ensure_period(sentence)}"
            except (ValueError, IndexError) as e:
                print(f"    ⚠️  Error processing numerical_coded value {value}: {e}")
                return None
    
    return None

def load_schema_csv_pairs(schema_folder: Path, data_folder: Path) -> List[Tuple[Dict, pd.DataFrame, str]]:
    """
    Load matching schema/CSV pairs.
    Returns: List of (schema_dict, dataframe, source_name) tuples
    """
    pairs = []
    
    # Get all schema files
    schema_files = sorted(schema_folder.glob("*_schema_master.json"))
    
    for schema_file in schema_files:
        # Derive CSV filename
        csv_name = schema_file.stem.replace("_schema_master", "") + ".csv"
        csv_path = data_folder / csv_name
        
        if not csv_path.exists():
            print(f"⚠️  Warning: No matching CSV for {schema_file.name}")
            continue
        
        # Load schema
        with open(schema_file, 'r', encoding='utf-8') as f:
            schema = json.load(f)
        
        # Load CSV
        df = pd.read_csv(csv_path)
        
        source_name = csv_name.replace(".csv", "")
        pairs.append((schema, df, source_name))
        print(f"✅ Loaded: {schema_file.name} + {csv_name}")
    
    return pairs

def generate_paragraph_for_row(row_index: int, schema_csv_pairs: List[Tuple[Dict, pd.DataFrame, str]]) -> Tuple[str, str, str]:
    """
    Generate a paragraph for a single row across all CSVs.
    Returns: (structure_id, coordinates, paragraph)
    """
    sentences = []
    structure_id = None
    coordinates = None
    
    for schema, df, source_name in schema_csv_pairs:
        row = df.iloc[row_index]
        
        # Extract structure_id and coordinates from first pair
        if structure_id is None:
            structure_id = row.iloc[0] if 'STRUCTURE_ID' not in df.columns else row['STRUCTURE_ID']
        if coordinates is None:
            coordinates = row.iloc[1] if 'COORDINATES' not in df.columns else row['COORDINATES']
        
        # Process each column (skip STRUCTURE_ID and COORDINATES)
        for column_name, schema_field in schema.items():
            if column_name in ['STRUCTURE_ID', 'COORDINATES']:
                continue
            
            # Skip reference and redundant types
            field_type = schema_field.get('type')
            if field_type in ['reference', 'numerical_redundant', 'nominal_redundant', 'numerical_coded_redundant']:
                continue
            
            if column_name not in df.columns:
                print(f"    ⚠️  Column mismatch: '{column_name}' in schema but not in {source_name}.csv")
                raise ValueError(f"Column mismatch: {column_name}")
            
            value = row[column_name]
            sentence = generate_sentence(column_name, value, schema_field)
            
            if sentence:
                sentences.append(sentence)
    
    # Combine all sentences into one paragraph
    paragraph = " ".join(sentences)
    
    return str(structure_id), str(coordinates), paragraph

def test_random_paragraph(schema_folder: str, data_folder: str, output_folder: str = "logs"):
    """
    Generate a paragraph for a random bridge and save to log file.
    """
    schema_folder = Path(schema_folder)
    data_folder = Path(data_folder)
    output_folder = Path(output_folder)
    output_folder.mkdir(exist_ok=True)
    
    print_section("TEST MODE - Random Paragraph Generation")
    
    # Load all pairs
    print("Loading schema/CSV pairs...")
    pairs = load_schema_csv_pairs(schema_folder, data_folder)
    
    if not pairs:
        print("❌ No valid schema/CSV pairs found")
        return
    
    # Get row count from first dataframe
    num_rows = len(pairs[0][1])
    print(f"\n📊 Total rows available: {num_rows:,}")
    
    # Pick random row
    random_row = random.randint(0, num_rows - 1)
    print(f"🎲 Selected random row index: {random_row}")
    
    # Generate paragraph
    print(f"\n{'─' * 80}")
    print("Generating paragraph...")
    print(f"{'─' * 80}\n")
    
    structure_id, coordinates, paragraph = generate_paragraph_for_row(random_row, pairs)
    
    # Save to log
    log_file = output_folder / f"test_paragraph_{structure_id}.txt"
    with open(log_file, 'w', encoding='utf-8') as f:
        f.write(f"STRUCTURE_ID: {structure_id}\n")
        f.write(f"COORDINATES: {coordinates}\n")
        f.write(f"ROW_INDEX: {random_row}\n")
        f.write(f"\n{'=' * 80}\n")
        f.write(f"PARAGRAPH:\n")
        f.write(f"{'=' * 80}\n\n")
        f.write(paragraph)
    
    print(f"✅ Saved to: {log_file}")
    print(f"\n📏 Paragraph length: {len(paragraph):,} characters")
    print(f"📝 Word count: {len(paragraph.split()):,} words")
    print(f"\n{'─' * 80}")
    print("Preview (first 500 characters):")
    print(f"{'─' * 80}")
    print(paragraph[:500] + "...\n")

def generate_all_paragraphs(schema_folder: str, data_folder: str, output_file: str):
    """
    Generate paragraphs for all bridges and save to CSV.
    """
    schema_folder = Path(schema_folder)
    data_folder = Path(data_folder)
    output_path = Path(output_file)
    
    print_section("GENERATING PARAGRAPHS FOR ALL BRIDGES")
    
    # Load all pairs
    print("Loading schema/CSV pairs...")
    pairs = load_schema_csv_pairs(schema_folder, data_folder)
    
    if not pairs:
        print("❌ No valid schema/CSV pairs found")
        return
    
    # Validate row counts match
    print("\n📊 Validating row counts...")
    row_counts = [len(df) for _, df, _ in pairs]
    if len(set(row_counts)) > 1:
        print("❌ ERROR: Row count mismatch across CSVs!")
        for schema, df, name in pairs:
            print(f"   {name}: {len(df):,} rows")
        raise ValueError("Row count mismatch")
    
    num_rows = row_counts[0]
    print(f"✅ All CSVs have {num_rows:,} rows")
    
    # Generate paragraphs
    print(f"\n{'=' * 80}")
    print("Generating paragraphs...")
    print(f"{'=' * 80}\n")
    
    results = []
    
    for row_idx in range(num_rows):
        if row_idx % 100 == 0:
            print(f"Progress: {row_idx:,} / {num_rows:,} ({100 * row_idx / num_rows:.1f}%)")
        
        try:
            structure_id, coordinates, paragraph = generate_paragraph_for_row(row_idx, pairs)
            results.append({
                'STRUCTURE_ID': structure_id,
                'COORDINATES': coordinates,
                'PARAGRAPH': paragraph
            })
        except Exception as e:
            print(f"❌ Error at row {row_idx}: {e}")
            raise
    
    # Save to CSV
    print(f"\n💾 Saving to {output_path}...")
    df_output = pd.DataFrame(results)
    df_output.to_csv(output_path, index=False, quoting=csv.QUOTE_ALL)
    
    print_section("COMPLETE")
    print(f"✅ Generated {len(results):,} paragraphs")

    print(f"💾 Saved to: {output_path}")
    print(f"   Min paragraph length: {df_output['PARAGRAPH'].str.len().min():,} characters")

    print(f"\n📏 Statistics:")
    print(f"   Min paragraph length: {df_output['PARAGRAPH'].str.len().min():,} characters")

    print(f"   Average paragraph length: {df_output['PARAGRAPH'].str.len().mean():.0f} characters")
    print(f"   Average paragraph length: {df_output['PARAGRAPH'].str.len().mean():.0f} characters")
    print(f"   Max paragraph length: {df_output['PARAGRAPH'].str.len().max():,} characters")

In [3]:
# Test mode - generate one random paragraph
test_random_paragraph(
    schema_folder=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\final_schemas",
    data_folder=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data_fixed"
)


                    TEST MODE - Random Paragraph Generation                     

Loading schema/CSV pairs...
✅ Loaded: design_maps_schema_master.json + design_maps.csv
✅ Loaded: macrostrat_schema_master.json + macrostrat.csv
✅ Loaded: nbi_nominal_schema_master.json + nbi_nominal.csv
✅ Loaded: nbi_numerical_coded_schema_master.json + nbi_numerical_coded.csv
✅ Loaded: nbi_numerical_schema_master.json + nbi_numerical.csv
✅ Loaded: nfhl_fema_flood_schema_master.json + nfhl_fema_flood.csv
✅ Loaded: nshm_hazard_grid_schema_master.json + nshm_hazard_grid.csv

📊 Total rows available: 4,914
🎲 Selected random row index: 456

────────────────────────────────────────────────────────────────────────────────
Generating paragraph...
────────────────────────────────────────────────────────────────────────────────

✅ Saved to: logs\test_paragraph_0005142A.txt

📏 Paragraph length: 21,808 characters
📝 Word count: 2,923 words

─────────────────────────────────────────────────────────────────────────────

In [6]:
# Full generation
generate_all_paragraphs(
    schema_folder=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\final_schemas",
    data_folder=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data_fixed",
    output_file=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\nlp_processing\nlp_data\bridge_paragraphs.csv"
)


                     GENERATING PARAGRAPHS FOR ALL BRIDGES                      

Loading schema/CSV pairs...
✅ Loaded: design_maps_schema_master.json + design_maps.csv
✅ Loaded: macrostrat_schema_master.json + macrostrat.csv
✅ Loaded: nbi_nominal_schema_master.json + nbi_nominal.csv
✅ Loaded: nbi_numerical_coded_schema_master.json + nbi_numerical_coded.csv
✅ Loaded: nbi_numerical_schema_master.json + nbi_numerical.csv
✅ Loaded: nfhl_fema_flood_schema_master.json + nfhl_fema_flood.csv
✅ Loaded: nshm_hazard_grid_schema_master.json + nshm_hazard_grid.csv

📊 Validating row counts...
✅ All CSVs have 4,914 rows

Generating paragraphs...

Progress: 0 / 4,914 (0.0%)
Progress: 100 / 4,914 (2.0%)
Progress: 200 / 4,914 (4.1%)
Progress: 300 / 4,914 (6.1%)
Progress: 400 / 4,914 (8.1%)
Progress: 500 / 4,914 (10.2%)
Progress: 600 / 4,914 (12.2%)
Progress: 700 / 4,914 (14.2%)
Progress: 800 / 4,914 (16.3%)
Progress: 900 / 4,914 (18.3%)
Progress: 1,000 / 4,914 (20.4%)
Progress: 1,100 / 4,914 (22.4%)
P

In [10]:
def get_numerical_label(value: float, edges: List, labels: List[str]) -> str:
    """
    Find which bin the value falls into and return corresponding label.
    Uses labels instead of sentences for snappier output.
    """
    # Convert "inf" strings to float('inf')
    edges_numeric = []
    for e in edges:
        if isinstance(e, str) and e.lower() in ['inf', '-inf']:
            edges_numeric.append(float(e))
        else:
            edges_numeric.append(float(e))
    
    # Find bin index
    bin_index = 0
    for i in range(len(edges_numeric) - 1):
        if value >= edges_numeric[i] and value < edges_numeric[i + 1]:
            bin_index = i
            break
        elif i == len(edges_numeric) - 2:  # Last bin (to infinity)
            bin_index = i
    
    return labels[bin_index]

def generate_sentence_snappy(column_name: str, value: Any, schema_field: Dict) -> str:
    """
    Generate a snappier sentence using labels instead of full sentences.
    Returns: "{title_snake_case}: {label}" in lowercase
    """
    # Skip null/missing values
    if pd.isna(value):
        return None
    
    title = schema_field.get('title', column_name)
    # Convert title to snake_case
    title_snake = title.replace(' ', '_')
    
    field_type = schema_field.get('type')
    
    if field_type == 'nominal':
        # Match value to code_map
        code_map = schema_field.get('code_map') or {}
        sentence = code_map.get(str(value))
        if sentence:
            return f"{title_snake}: {ensure_period(sentence)}".lower()
    
    elif field_type == 'nl':
        # Use raw value as sentence (unchanged)
        return f"{title_snake}: {ensure_period(str(value))}".lower()
    
    elif field_type == 'numerical':
        # Use labels instead of sentences
        semantic_bins = schema_field.get('semantic_bins', {})
        edges = semantic_bins.get('edges', [])
        labels = semantic_bins.get('labels', [])
        
        if edges and labels:
            try:
                label = get_numerical_label(float(value), edges, labels)
                return f"{title_snake}: {ensure_period(label)}".lower()
            except (ValueError, IndexError) as e:
                print(f"    ⚠️  Error processing numerical value {value}: {e}")
                return None
    
    elif field_type == 'numerical_coded':
        # Check code_map first - if value is in code_map, skip it
        code_map = schema_field.get('code_map') or {}
        if str(value) in code_map or (isinstance(value, (int, float)) and str(int(value)) in code_map):
            return None  # Skip coded values
        
        # Otherwise use labels like numerical
        semantic_bins = schema_field.get('semantic_bins', {})
        edges = semantic_bins.get('edges', [])
        labels = semantic_bins.get('labels', [])
        
        if edges and labels:
            try:
                label = get_numerical_label(float(value), edges, labels)
                return f"{title_snake}: {ensure_period(label)}".lower()
            except (ValueError, IndexError) as e:
                print(f"    ⚠️  Error processing numerical_coded value {value}: {e}")
                return None
    
    return None

def generate_paragraph_snappy(row_index: int, schema_csv_pairs: List[Tuple[Dict, pd.DataFrame, str]]) -> Tuple[str, str, str]:
    """
    Generate a snappier paragraph for a single row across all CSVs.
    Returns: (structure_id, coordinates, paragraph)
    """
    sentences = []
    structure_id = None
    coordinates = None
    
    for schema, df, source_name in schema_csv_pairs:
        row = df.iloc[row_index]
        
        # Extract structure_id and coordinates from first pair
        if structure_id is None:
            structure_id = row.iloc[0] if 'STRUCTURE_ID' not in df.columns else row['STRUCTURE_ID']
        if coordinates is None:
            coordinates = row.iloc[1] if 'COORDINATES' not in df.columns else row['COORDINATES']
        
        # Process each column (skip STRUCTURE_ID and COORDINATES)
        for column_name, schema_field in schema.items():
            if column_name in ['STRUCTURE_ID', 'COORDINATES']:
                continue
            
            # Skip reference and redundant types
            field_type = schema_field.get('type')
            if field_type in ['reference', 'numerical_redundant', 'nominal_redundant', 'numerical_coded_redundant']:
                continue
            
            if column_name not in df.columns:
                print(f"    ⚠️  Column mismatch: '{column_name}' in schema but not in {source_name}.csv")
                raise ValueError(f"Column mismatch: {column_name}")
            
            value = row[column_name]
            sentence = generate_sentence_snappy(column_name, value, schema_field)
            
            if sentence:
                sentences.append(sentence)
    
    # Combine all sentences into one paragraph
    paragraph = " ".join(sentences)
    
    return str(structure_id), str(coordinates), paragraph

def generate_all_paragraphs_snappy(schema_folder: str, data_folder: str, output_file: str):
    """
    Generate snappier paragraphs for all bridges and save to CSV.
    """
    schema_folder = Path(schema_folder)
    data_folder = Path(data_folder)
    output_path = Path(output_file)
    
    print_section("GENERATING SNAPPY PARAGRAPHS FOR ALL BRIDGES")
    
    # Load all pairs
    print("Loading schema/CSV pairs...")
    pairs = load_schema_csv_pairs(schema_folder, data_folder)
    
    if not pairs:
        print("❌ No valid schema/CSV pairs found")
        return
    
    # Validate row counts match
    print("\n📊 Validating row counts...")
    row_counts = [len(df) for _, df, _ in pairs]
    if len(set(row_counts)) > 1:
        print("❌ ERROR: Row count mismatch across CSVs!")
        for schema, df, name in pairs:
            print(f"   {name}: {len(df):,} rows")
        raise ValueError("Row count mismatch")
    
    num_rows = row_counts[0]
    print(f"✅ All CSVs have {num_rows:,} rows")
    
    # Generate paragraphs
    print(f"\n{'=' * 80}")
    print("Generating snappy paragraphs...")
    print(f"{'=' * 80}\n")
    
    results = []
    
    for row_idx in range(num_rows):
        if row_idx % 100 == 0:
            print(f"Progress: {row_idx:,} / {num_rows:,} ({100 * row_idx / num_rows:.1f}%)")
        
        try:
            structure_id, coordinates, paragraph = generate_paragraph_snappy(row_idx, pairs)
            results.append({
                'STRUCTURE_ID': structure_id,
                'COORDINATES': coordinates,
                'PARAGRAPH': paragraph
            })
        except Exception as e:
            print(f"❌ Error at row {row_idx}: {e}")
            raise
    
    # Save to CSV
    print(f"\n💾 Saving to {output_path}...")
    df_output = pd.DataFrame(results)
    df_output.to_csv(output_path, index=False, quoting=csv.QUOTE_ALL)
    
    print_section("COMPLETE")
    print(f"✅ Generated {len(results):,} paragraphs")
    print(f"💾 Saved to: {output_path}")
    print(f"\n📏 Statistics:")
    print(f"   Average paragraph length: {df_output['PARAGRAPH'].str.len().mean():.0f} characters")
    print(f"   Max paragraph length: {df_output['PARAGRAPH'].str.len().max():,} characters")
    print(f"   Min paragraph length: {df_output['PARAGRAPH'].str.len().min():,} characters")


In [11]:
generate_all_paragraphs_snappy(
    schema_folder=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\nlp_processing\schema_for_short",
    data_folder=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data_fixed",
    output_file=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\nlp_processing\sentence_data\sentences_short.csv"
)


                  GENERATING SNAPPY PARAGRAPHS FOR ALL BRIDGES                  

Loading schema/CSV pairs...
✅ Loaded: design_maps_schema_master.json + design_maps.csv
✅ Loaded: macrostrat_schema_master.json + macrostrat.csv
✅ Loaded: nbi_nominal_schema_master.json + nbi_nominal.csv
✅ Loaded: nbi_numerical_coded_schema_master.json + nbi_numerical_coded.csv
✅ Loaded: nbi_numerical_schema_master.json + nbi_numerical.csv
✅ Loaded: nfhl_fema_flood_schema_master.json + nfhl_fema_flood.csv
✅ Loaded: nshm_hazard_grid_schema_master.json + nshm_hazard_grid.csv

📊 Validating row counts...
✅ All CSVs have 4,914 rows

Generating snappy paragraphs...

Progress: 0 / 4,914 (0.0%)
Progress: 100 / 4,914 (2.0%)
Progress: 200 / 4,914 (4.1%)
Progress: 300 / 4,914 (6.1%)
Progress: 400 / 4,914 (8.1%)
Progress: 500 / 4,914 (10.2%)
Progress: 600 / 4,914 (12.2%)
Progress: 700 / 4,914 (14.2%)
Progress: 800 / 4,914 (16.3%)
Progress: 900 / 4,914 (18.3%)
Progress: 1,000 / 4,914 (20.4%)
Progress: 1,100 / 4,914 (2

In [5]:
def update_nominal_codemaps(current_schema_path: str, old_schema_path: str, output_schema_path: str):
    """
    Update nominal code_maps in current schema with code_maps from old schema.
    Finds corresponding columns and replaces code_map for nominal type fields.
    """
    # Load schemas
    with open(current_schema_path, 'r', encoding='utf-8') as f:
        current_schema = json.load(f)
    
    with open(old_schema_path, 'r', encoding='utf-8') as f:
        old_schema = json.load(f)
    
    print(f"📖 Loaded current schema: {Path(current_schema_path).name}")
    print(f"📖 Loaded old schema: {Path(old_schema_path).name}")
    print()
    
    updated_count = 0
    
    # Iterate through current schema
    for column_name, field_info in current_schema.items():
        # Check if field is nominal type
        if field_info.get('type') == 'nominal':
            # Check if column exists in old schema
            if column_name in old_schema:
                old_field = old_schema[column_name]
                
                # Check if old schema has code_map
                if 'code_map' in old_field:
                    # Replace code_map
                    current_schema[column_name]['code_map'] = old_field['code_map']
                    updated_count += 1
                    print(f"✅ Updated '{column_name}' code_map")
                else:
                    print(f"⚠️  No code_map in old schema for '{column_name}'")
            else:
                print(f"⚠️  Column '{column_name}' not found in old schema")
    
    # Save updated schema
    with open(output_schema_path, 'w', encoding='utf-8') as f:
        json.dump(current_schema, f, indent=2)
    
    print()
    print(f"{'='*60}")
    print(f"✅ Updated {updated_count} nominal code_maps")
    print(f"💾 Saved to: {output_schema_path}")
    print(f"{'='*60}")

In [6]:
update_nominal_codemaps(
    current_schema_path=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\nlp_processing\schema_for_short\nbi_nominal_schema_master.json",
    old_schema_path=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\phase_1_schemas\nbi_nominal_schema.json",
    output_schema_path=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\nlp_processing\schema_for_short\nbi_nominal_schema_master.json"
)

📖 Loaded current schema: nbi_nominal_schema_master.json
📖 Loaded old schema: nbi_nominal_schema.json

✅ Updated 'STATE_CODE_001' code_map
✅ Updated 'COUNTY_CODE_003' code_map
✅ Updated 'HIGHWAY_DISTRICT_002' code_map
✅ Updated 'ROUTE_PREFIX_005B' code_map
✅ Updated 'SERVICE_LEVEL_005C' code_map
✅ Updated 'BASE_HWY_NETWORK_012' code_map
✅ Updated 'TOLL_020' code_map
✅ Updated 'MAINTENANCE_021' code_map
✅ Updated 'FUNCTIONAL_CLASS_026' code_map
✅ Updated 'DESIGN_LOAD_031' code_map
✅ Updated 'MEDIAN_CODE_033' code_map
✅ Updated 'STRUCTURE_FLARED_035' code_map
✅ Updated 'RAILINGS_036A' code_map
✅ Updated 'TRANSITIONS_036B' code_map
✅ Updated 'APPR_RAIL_036C' code_map
✅ Updated 'APPR_RAIL_END_036D' code_map
✅ Updated 'HISTORY_037' code_map
✅ Updated 'NAVIGATION_038' code_map
✅ Updated 'OPEN_CLOSED_POSTED_041' code_map
✅ Updated 'SERVICE_ON_042A' code_map
✅ Updated 'SERVICE_UND_042B' code_map
✅ Updated 'STRUCTURE_KIND_043A' code_map
✅ Updated 'STRUCTURE_TYPE_043B' code_map
✅ Updated 'APPR_KI

In [ ]:
# manual edits
"""
"NFHL_FLD_ZONE": {
    "type": "nominal",
    "description": "The flood hazard zone classification describes the level and type of flood risk assigned to the location based on FEMA floodplain mapping.",
    "code_map": {
      "A": "high risk no elevation data",
      "AE": "high risk with elevation data",
      "AO": "shallow flooding sheet flow",
      "AH": "shallow flooding with elevation",
      "V": "coastal high velocity zone",
      "VE": "coastal velocity with elevation",
      "X": "minimal flood risk",
      "X (0.2%)": "moderate risk low probability",
      "D": "undetermined flood hazard"
    },
    "title": "Flood Hazard Zone"
  }
"""
"""
  "NFHL_SFHA": {
    "type": "nominal",
    "description": "The indicator of Special Flood Hazard Area status describes whether the location is subject to a 1-percent-annual-chance (100-year) flood hazard.",
    "code_map": {
      "T": "within special flood hazard area",
      "F": "outside special flood hazard area"
    },
    "title": "Flood Hazard Status"
  },
"""
"""
  "SDCS": {
    "type": "nominal",
    "description": "The seismic design category that classifies overall seismic risk at the site and governs required detailing, analysis rigor, and design provisions.",
    "code_map": {
      "A": "very low seismic hazard",
      "B": "low seismic hazard",
      "C": "moderate seismic hazard",
      "D": "high seismic hazard",
      "E": "very high seismic hazard",
      "F": "extreme site specific hazard"
    },
    "title": "Seismic Design Category"
  },
"""

''

In [9]:
def test_random_paragraph_snappy(schema_folder: str, data_folder: str, output_folder: str = "logs"):
    """
    Generate a snappy paragraph for a random bridge and save to log file.
    """
    schema_folder = Path(schema_folder)
    data_folder = Path(data_folder)
    output_folder = Path(output_folder)
    output_folder.mkdir(exist_ok=True)
    
    print_section("TEST MODE - Random Snappy Paragraph Generation")
    
    # Load all pairs
    print("Loading schema/CSV pairs...")
    pairs = load_schema_csv_pairs(schema_folder, data_folder)
    
    if not pairs:
        print("❌ No valid schema/CSV pairs found")
        return
    
    # Get row count from first dataframe
    num_rows = len(pairs[0][1])
    print(f"\n📊 Total rows available: {num_rows:,}")
    
    # Pick random row
    random_row = random.randint(0, num_rows - 1)
    print(f"🎲 Selected random row index: {random_row}")
    
    # Generate paragraph
    print(f"\n{'─' * 80}")
    print("Generating snappy paragraph...")
    print(f"{'─' * 80}\n")
    
    structure_id, coordinates, paragraph = generate_paragraph_snappy(random_row, pairs)
    
    # Save to log
    log_file = output_folder / f"test_paragraph_snappy_{structure_id}.txt"
    with open(log_file, 'w', encoding='utf-8') as f:
        f.write(f"STRUCTURE_ID: {structure_id}\n")
        f.write(f"COORDINATES: {coordinates}\n")
        f.write(f"ROW_INDEX: {random_row}\n")
        f.write(f"\n{'=' * 80}\n")
        f.write(f"SNAPPY PARAGRAPH:\n")
        f.write(f"{'=' * 80}\n\n")
        f.write(paragraph)
    
    print(f"✅ Saved to: {log_file}")
    print(f"\n📏 Paragraph length: {len(paragraph):,} characters")
    print(f"📝 Word count: {len(paragraph.split()):,} words")
    print(f"\n{'─' * 80}")
    print("Preview (first 500 characters):")
    print(f"{'─' * 80}")
    print(paragraph[:500] + "...\n")

# Test mode - generate one random snappy paragraph
test_random_paragraph_snappy(
    schema_folder=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\nlp_processing\schema_for_short",
    data_folder=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data_fixed"
)


                 TEST MODE - Random Snappy Paragraph Generation                 

Loading schema/CSV pairs...
✅ Loaded: design_maps_schema_master.json + design_maps.csv
✅ Loaded: macrostrat_schema_master.json + macrostrat.csv
✅ Loaded: nbi_nominal_schema_master.json + nbi_nominal.csv
✅ Loaded: nbi_numerical_coded_schema_master.json + nbi_numerical_coded.csv
✅ Loaded: nbi_numerical_schema_master.json + nbi_numerical.csv
✅ Loaded: nfhl_fema_flood_schema_master.json + nfhl_fema_flood.csv
✅ Loaded: nshm_hazard_grid_schema_master.json + nshm_hazard_grid.csv

📊 Total rows available: 4,914
🎲 Selected random row index: 386

────────────────────────────────────────────────────────────────────────────────
Generating snappy paragraph...
────────────────────────────────────────────────────────────────────────────────

✅ Saved to: logs\test_paragraph_snappy_0004328B.txt

📏 Paragraph length: 4,049 characters
📝 Word count: 373 words

──────────────────────────────────────────────────────────────────